
# Phylogenetics Pipeline

This notebook demonstrates a full phylogenetics pipeline using the `phylogenetics` package. In this example, it is assumed that we've already downloaded a set of homologs from NCBI blast and converted them into a `HomologSet` object. To see how to do this, read the wiki on the Github page.

In [1]:
from phylogenetics.homologs import load_homologset

hs = load_homologset("pickles/01-ubq-sequences.pickle")

## `HomologSet` object

The `HomologSet` object in the phylogenetics package is a persistent data-structure for holding, processing, writing, and analyzing a set of homologs in a phylogenetics project. Inside this set are `Homolog` Objects, which hold the metdata of a sequence in the set. 

**NOTE**: each homolog is given a unique 10 character id.

We can print it out to many different formats, including `.pickle`, `.fasta`, `.csv`, `.phylip`, and `.json`.

We can look at a single homolog:

In [2]:
hs.homologs[0].__dict__

{'accession': '3TGD_A',
 'blast_query': 'gi|51338685|sp|P62837.1|UB2D2_HUMAN',
 'defline': 'Chain A, Crystal Structure Of The Human Ubiquitin-Conjugating Enzyme (E2) Ubch5b',
 'evalue': '1.81911e-105',
 'gid': 'gi|345111056|pdb|3TGD|A',
 'id': 'XX00000000',
 'len': '152',
 'sequence': 'GAMGSMALKRIHKELNDLARDPPAQCSAGPVGDDMFHWQATIMGPNDSPYQGGVFFLTIHFPTDYPFKPPKVAFTTRIYHPNINSNGSICLDILRSQWSPALTISKVLLSICSLLCDPNPDDPLVPEIARIYKTDREKYNRIAREWTQKYAM'}

We can also print an individual `Homolog` in the formats mentioned above.

In [3]:
hs.homologs[0].csv()

'sequence,evalue,blast_query,accession,len,defline,gid,idGAMGSMALKRIHKELNDLARDPPAQCSAGPVGDDMFHWQATIMGPNDSPYQGGVFFLTIHFPTDYPFKPPKVAFTTRIYHPNINSNGSICLDILRSQWSPALTISKVLLSICSLLCDPNPDDPLVPEIARIYKTDREKYNRIAREWTQKYAM,1.81911e-105,gi|51338685|sp|P62837.1|UB2D2_HUMAN,3TGD_A,152,Chain A, Crystal Structure Of The Human Ubiquitin-Conjugating Enzyme (E2) Ubch5b,gi|345111056|pdb|3TGD|A,XX00000000\n'

In [4]:
hs.homologs[0].json()

'{"sequence": "GAMGSMALKRIHKELNDLARDPPAQCSAGPVGDDMFHWQATIMGPNDSPYQGGVFFLTIHFPTDYPFKPPKVAFTTRIYHPNINSNGSICLDILRSQWSPALTISKVLLSICSLLCDPNPDDPLVPEIARIYKTDREKYNRIAREWTQKYAM", "evalue": "1.81911e-105", "blast_query": "gi|51338685|sp|P62837.1|UB2D2_HUMAN", "accession": "3TGD_A", "len": "152", "defline": "Chain A, Crystal Structure Of The Human Ubiquitin-Conjugating Enzyme (E2) Ubch5b", "gid": "gi|345111056|pdb|3TGD|A", "id": "XX00000000"}'

In [5]:
hs.homologs[0].fasta()

'>XX00000000\nGAMGSMALKRIHKELNDLARDPPAQCSAGPVGDDMFHWQATIMGPNDSPYQGGVFFLTIHFPTDYPFKPPKVAFTTRIYHPNINSNGSICLDILRSQWSPALTISKVLLSICSLLCDPNPDDPLVPEIARIYKTDREKYNRIAREWTQKYAM\n'

## Cluster sequences

`phylogenetics` package includes a wrapper around `cdhit`, exposing a simple python API to cluster sequences. This program allows us to cluster sequences that are too similar by defining a cutoff fraction. We can also define accession numbers and species names that we don't want to bury in clusters (see docstring).

In [8]:
from phylogenetics.cdhit import run_cdhit

# Define redundancy cutoff
cutoff = 0.99

# Run cdhit on homologset
hs2 = run_cdhit(hs, redund_cutoff=cutoff)

# print the new length of the homologset
print("Number of clusters:")
print(len(hs2.homologs))

# Write this to a file.
hs2.write("pickles/02-clustered.pickle", format="pickle")

Number of clusters:
34


# Multiple Sequence Alignment

Using `MSAProbs`, we can also construct a multiple sequence alignment. This will attach a new attribute to each `Homolog` that describes the lastest alignment made. You can add new alignments, if you manually align the homologs, using the `alignment_to_homologset` method.

In [9]:
from phylogenetics.msaprobs import run_msaprobs

# Run the Multiple sequence alignment
hs3 = run_msaprobs(hs2)

# Write this to a file
hs3.write("pickles/03-aligned.pickle", format="pickle")

# Build a Tree

Once we've built an alignment, we can construct a tree from the `HomologSet` using PhyML. This will generate a phylip file on the fly, run through PhyML, and add the tree as an attribute to the `HomologSet`. It will also output a `.nwk` file containing the tree, which can be viewed through any tree viewer.

In [11]:
from phylogenetics.phyml import run_phyml

# Run PhyML
hs4 = run_phyml(hs3, "04-tree") 

# Write the new homologset to file
hs4.write("pickles/04-tree.pickles", format="pickle")